In [3]:
## Libraries
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
import pv_model_class
import pandas as pd

In [4]:
# Constants and variables

Iscn = 8.21 #8.7 # nominal short circuit current
Vocn = 32.9 #37.7 # nominal open circuit voltage
Imp = 7.61 #8.2 # array current at MPP
Vmp = 26.3 #30.1 # array voltage at MPP
Pmax_e = Vmp * Imp # experimental array max power output
Kv = -0.123 #-0.32/100 * Vocn # voltage temperature coefficient
Ki = 0.0032 #-0.032/100 * Iscn # current temperature coefficient
Ns = 54 #60. # number of series cells
Gn = 1000. # nominal irradiance
G = Gn
Tn = 25. + 273.15 # nominal operating temperature
T = Tn

Egap = 1.8 * 10 ** -19 # Bandgap of silicon (silicio cristalino)

err = 0.0001
inverter = 0.95
array_dim = [10, 10]

In [5]:
df = pd.read_csv('/Users/margotadam/Downloads/43386d97fd77fc514fa97f6166756b56/242127_47.69_-121.10_2015.csv')
df.head()

/Users/margotadam/opt/anaconda3/envs/real_modeling/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,41,42,43,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,...,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
0,NSRDB,242127,-,-,-,47.69,-121.1,-8,1308,-8,...,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,NaN,3.0.6
1,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Wind Speed,Relative Humidity,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,0,0,0,0,0,0.8,70.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,0,30,0,0,0,0.8,70.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,1,0,0,0,0,0.9,70.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
new_header = df.iloc[1]
tmy = df[2:]
tmy.columns = new_header
tmy = tmy.dropna(axis=1)
tmy['Hour'][30]

'14'

In [7]:
tmy_july = tmy[(tmy['Month'] == '7') & (tmy['Day'] == '3')]
tmy_jan = tmy[(tmy['Month'] == '1') & (tmy['Day'] == '2')]
tmy_jan

1,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Wind Speed,Relative Humidity,Temperature
50,2015,1,2,0,0,0,0,0,0.5,79.27,-9
51,2015,1,2,0,30,0,0,0,0.4,79.25,-8
52,2015,1,2,1,0,0,0,0,0.30000000000000004,81.85000000000001,-8
53,2015,1,2,1,30,0,0,0,0.2,75.74,-7
54,2015,1,2,2,0,0,0,0,0.2,78.03,-7
55,2015,1,2,2,30,0,0,0,0.4,78.01,-7
56,2015,1,2,3,0,0,0,0,0.6000000000000001,76.16,-7
57,2015,1,2,3,30,0,0,0,0.7000000000000001,76.14,-7
58,2015,1,2,4,0,0,0,0,0.9,78.27,-8
59,2015,1,2,4,30,0,0,0,0.8,78.26,-8


In [11]:
### Equations from Masters textbook (table 4.1 and table 4.2)
DNI = int(tmy['DNI'][12795])
DHI = int(tmy['DHI'][12795])
GHI = int(tmy['GHI'][12795])

# ground reflectance
rho = 0.2
# number of days
#month = int(tmy['Month'][12795])
#day = int(tmy['Day'][12795])
#n = (30*month - 30) + day

# calculated by hand for 9/24
n = 267
# hour
hour = int(tmy['Hour'][12795])
# latitude
lat = 47.69
# collector azimuth
az_c = 0
# collector tilt angle
sigma = lat * 0.76 + 3.1
# declination
decl = 23.45 * np.sin(360/365 * (n - 81))
# hour angle
H = 15 * (12 - hour)
# solar altitude
beta = np.arcsin(np.cos(lat) * np.cos(decl) * np.cos(H) + np.sin(lat) * np.sin(decl))
# solar azimuth
az_s = np.arcsin(np.cos(decl) * np.sin(H) / np.cos(beta))

cos_theta = np.cos(beta) * np.cos(az_s - az_c) * np.sin(sigma) + np.sin(beta) * np.cos(sigma)
B = DNI * cos_theta
D = DHI * (1 + np.cos(sigma)) / 2
R = GHI * rho * (1 - np.cos(sigma)) / 2
G = B + D + R
print(G)

250.4324849782308


In [8]:
## calculating PV model for consecutive hours throughout a day

pv_mod = pv_model_class.PvModel(Iscn, Vocn, Imp, Vmp, Kv, Ki, Ns, Gn, G, Tn, T, Egap, err, inverter, array_dim)
# find resistor values and diode ideality constant
Rs, Rp, a = pv_mod.find_resistors()
# find voltage, current, and power arrays
v, i, P = pv_mod.calculate_power(Rs, Rp, a)

max_power = []

# calculate day number
#    7/3 -- 184
n = 2
# ground reflectance
rho = 0.2
# latitude
lat = 47.69
# collector azimuth
az_c = 0
# collector tilt angle
sigma = lat * 0.76 + 3.1

for i in range(2, 24*2 + 2, 2):
    # shift to get to month of July 3rd
    # i += 8786
    
    # shift to get to Jan 2nd
    i += 50
    hour = int(tmy['Hour'][i])
    print(tmy['Month'][i], tmy['Day'][i], tmy['Hour'][i])
    DNI = int(tmy['DNI'][i])
    DHI = int(tmy['DHI'][i])
    GHI = int(tmy['GHI'][i])
    pv_mod.G = pv_mod.find_irradiance(DNI, DHI, GHI, rho, n, hour, lat, az_c, sigma)
    print(pv_mod.G)
    v, i, P = pv_mod.calculate_power(Rs, Rp, a)
    max_idx = np.argmax(P)
    if P[max_idx] < 0.001:
        P[max_idx] = 0
    max_power.append(P[max_idx])

print(max_power)
print(len(max_power))

## if pv array of 3 strings with 2 modules in series per string (6 total)
max_power_array = max_power * 6

1 2 1
0.0
1 2 2
0.0
1 2 3
0.0
1 2 4
0.0
1 2 5
0.0
1 2 6
0.0
1 2 7
0.0
1 2 8
0.0
1 2 9
3.6373586464014624
1 2 10
119.98241829487228
1 2 11
103.28148239878854
1 2 12
390.58464777690943
1 2 13
150.37306769029564
1 2 14
107.7428739237553
1 2 15
160.48622151029033
1 2 16
27.28018984801097
1 2 17
0.0
1 2 18
0.0
1 2 19
0.0
1 2 20
0.0
1 2 21
0.0
1 2 22
0.0
1 2 23
0.0
1 3 0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.687437411241854, 1887.4678324745719, 1569.092473952668, 7204.08116258169, 2473.208104704921, 1653.8375698519135, 2669.471806669538, 205.99573028765283, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
24
